In [ ]:
import requests
import time
import json
from google.colab import files

def callapi(fileArray, api_key, blog1, blog2):
    start_time = time.time()
    url = "https://api.worqhat.com/api/ai/content/v4"
    payload = {
        "question": f"I am providing you with two blog entries that I previously wrote and some audio files that discuss my current thought process. Based on the audio, please draft my next blog post of exact or above 500 words. Ensure that the new blog aligns with the writing style of my previous blogs while incorporating the ideas from the audio recordings.Add Subheadings and make it as attractive to the reader as possible .Here are the blogs:\n\nBlog 1: {blog1}\n\nBlog 2: {blog2}",
        "training_data": f"Analyze the provided audio files and blog content.Create a cohesive blog post that expands on the thought process from the audio while maintaining a similar writing style to the provided blogs. Return the result in markdown format with one heading and subheadings and keep the key as blog_post. ",
        "response_type": "json",
        "model": "aicon-v4-nano-160824",
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
    }

    print("The request is being sent...")
    response = requests.request(
        "POST", url, headers=headers, data=payload, files=fileArray)
    end_time = time.time()  
    duration = end_time - start_time  
    if response.status_code == 200:
        return response.json(), round(duration, 2)
    else:
        print("The request failed.")
        return None, round(duration, 2)


In [ ]:
# Get API key from user
api_key = input("Enter your WorqHat API Key: ")

# Get previous blog entries
blog1 = input("Enter your first blog here: ")
blog2 = input("Enter your second blog here: ")

# Get audio files using Google Colab upload feature
print("Upload audio files discussing your thought process (mp3, wav, ogg):")
uploaded_audios = files.upload()

# Create fileArray with the desired format
fileArray = [
    ('files', (file_name, file_content, 'audio/mpeg')) 
    for file_name, file_content in uploaded_audios.items()
]

# Call the API
if fileArray:
    response_v2, duration_v2 = callapi(fileArray, api_key, blog1, blog2)
    if response_v2:
        print(response_v2)
        content = response_v2.get('content', '')
        try:
            parsed_content = json.loads(content)
            print(f"✅ Blog generated successfully! (Time taken: {duration_v2} seconds)")
            print(parsed_content.get('blog_post', ''))
        except json.JSONDecodeError:
            print("❌ Failed to parse the response content. Please check the API response.")
    else:
        print("❌ Failed to generate the blog post.")
else:
    print("No audio files uploaded.")